! jt -r

In [53]:
from importlib import reload
reload(aux)
reload(con)
reload(info)
reload(graph)

<module 'src.graphic' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\graphic.py'>

In [2]:
import pandas as pd
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime,date
import src.info as info
import src.in_out as io
import numpy as np
import itertools
import src.graphic as graph

# Hazırlık Aşaması

**data** klasöründeki tüm dosya yolları alınıyor
```python
>>> path_list
Out []:
 ['C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180906_6C_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6A_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6B_U8.xlsx',
 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\20180907_6C_U8.xlsx']
```

In [3]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

Ürün adları sırayla alınıyor
```py
>>> product_names
Out[]:
['6AU8', '6BU8', '6CU8', '6AU8', '6BU8', '6CU8']```

In [4]:
product_names = [info.get_productName(path) for path in path_list]

Tarihler sırayla alınıyor
```py
>>> date_list
Out[]:
[Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00'),
 Timestamp('2018-09-07 00:00:00')]
```

In [5]:
date_list = [info.get_productDate(path) for path in path_list]

Hangi sütunlardaki verileri alacağımızı aşağıdaki listede belirityoruz

In [6]:
cols= ["Time","BID price","ASK price"]

Dosyalardan veriler dataframe olarak okunuyor ve listeye atılıyor.
Okunan_veriler'in yapısı: <br/>
```py
list(pd.DataFrame,pd.DataFrame.....)
```

In [7]:
okunan_veriler = [pd.read_excel(path,usecols=cols) for path in path_list]

Her bir DataFrame'deki sütunlar yeniden adlandırılıyor

In [15]:
columns=['time','bid_price','ask_price']

In [16]:
for df in okunan_veriler:
    df.columns = columns

In [17]:
time_stamp = okunan_veriler[0].time

# Mid Price Hesaplama ve Saatlik Bölme 

hesaplanan her bir mid_price **list_of_mid_price** adlı listeye atılıyor. Listenin yapısı aşağıdaki gibidir:
```py
list(pd.Series,pd.Series,pd.Series...)
```

In [20]:
list_of_mid_price = list(map(aux.get_mid_price,okunan_veriler))

Bulunan mid price hiyerarşik indeksleme ile tek DataFrame'de birleştiriliyor.

In [22]:
arrays = [date_list,product_names]

In [23]:
df_mp = pd.DataFrame(data=list_of_mid_price, index = arrays)

Bu dataframe zaman sütunu eklenip saatlik olarak bölünüyor.

In [25]:
df_mp_t = df_mp.T

In [27]:
df_mp_t['time'] = time_stamp

```py
>>> len(list_of_hourly_mp)
Out[]: 23
>>> df.columns.levels
Out[]:FrozenList([[2018-09-06 00:00:00, 2018-09-07 00:00:00, 'time'], ['6AU8', '6BU8', '6CU8', '']])
```

In [29]:
list_of_hourly_mp = aux.split_df(df_mp_t,23)

----

# Ürünler Eşleştiriliyor

```python
>>> non_duplicate_dates
Out[]:[Timestamp('2018-09-06 00:00:00'), Timestamp('2018-09-07 00:00:00')]
>>> non_duplicate_names
Out[]:['6AU8', '6BU8', '6CU8']
>>> paired_products
Out[]: [('6AU8', '6BU8'), ('6AU8', '6CU8'), ('6BU8', '6CU8')]
```

In [41]:
non_duplicate_dates = list(dict.fromkeys(date_list)) #tekrar eden tarihler kaldırıldı.

In [42]:
non_duplicate_names = list(dict.fromkeys(product_names)) #tekrar eden ürünler kaldırıldı.

In [43]:
paired_products = list(itertools.combinations(non_duplicate_names, 2)) #2'li kombinasyon kullanılarak ürünler eşleştiriliyor.

----

# Spread Hesaplanıyor

**list_of_hourly_mp** da hiyerarşi şu şekildedir: <br/>
* Önce saate erişiriz
* İkinci olarak tarihe erişiriz
* Son olarak ürüne erişiriz <br/>
**Buna göre spread hesaplamak için aşağıdaki yol izlenir:** <br/>
    1. Her bir saate eriş
    2. Her bir tarihe eriş
    3. Pairleri al
    4. *spread_hesapla(pair): return pd.Series*
    

In [80]:
values={'a_PNLTICK':10,
'a_TICKSIZE':0.0001,
'b_PNLTICK':6.25,
'b_TICKSIZE':0.0001}

In [66]:
list_of_hourly_mp=x

In [78]:
def spread_al(df):
    spread_dict = {}
    for date in non_duplicate_dates:
        for pair in paired_products:
            mp_1 = df[pd.Timestamp(date)][pair[0]]
            mp_2 = df[pd.Timestamp(date)][pair[1]]
            spread_dict[(date,pair[0]+"_"+pair[1])] = aux.find_spread(mp_1,mp_2,values)
    new_df = pd.DataFrame(spread_dict)
    new_df['time'] = df.time
    return new_df

In [81]:
spread = [spread_al(df) for df in list_of_hourly_mp]

In [84]:
list_of_hourly_mp[0]

2018-09-06 00:00:00                    2018-09-07 00:00:00           \
                    6AU8      6BU8     6CU8                6AU8     6BU8   
0               0.694125  1.357794  0.71930            1.424213  1.74155   
1               0.694128  1.357794  0.71930            1.563086  1.24155   
2               0.694078  1.357794  0.71925            1.922686  1.24155   
3               0.694078  1.357853  0.71925            1.929929  1.24160   
4               0.694078  1.416903  0.71925            1.070329  1.29165   
...                  ...       ...      ...                 ...      ...   
3595            0.694078  1.357685  0.71925            0.809213  1.24145   
3596            0.694078  1.357685  0.71925            0.809213  1.24145   
3597            0.694078  1.357685  0.71925            0.809213  1.24145   
3598            0.694078  1.357685  0.71925            0.809213  1.24145   
3599            0.694078  1.357685  0.71925            0.809213  1.24145   

                   time  
         6CU8            
0     0.71930  18:00:00  
1     0.71930  18:00:01  
2     0.71925  18:00:02  
3     0.71925  18:00:03  
4     0.71925  18:00:04  
...       ...       ...  
3595  0.71925  18:59:55  
3596  0.71925  18:59:56  
3597  0.71925  18:59:57  
3598  0.71925  18:59:58  
3599  0.71925  18:59:59  

[3600 rows x 7 columns]

In [83]:
spread[0]

2018-09-06 00:00:00                     2018-09-07 00:00:00             \
               6AU8_6BU8 6AU8_6CU8 6BU8_6CU8           6AU8_6BU8  6AU8_6CU8   
0                 0.0000     0.000     0.000               0.000      0.000   
1                 0.3500     0.350     0.000           45137.370  13887.370   
2                -4.6500    -1.525     3.125           81097.370  49850.495   
3                -8.3375    -1.525     9.025           81818.545  50574.795   
4             -3698.9625    -1.525  5914.025           -7269.580 -35385.205   
...                  ...       ...       ...                 ...        ...   
3595              2.1625    -1.525    -7.775          -30243.750 -61496.875   
3596              2.1625    -1.525    -7.775          -30243.750 -61496.875   
3597              2.1625    -1.525    -7.775          -30243.750 -61496.875   
3598              2.1625    -1.525    -7.775          -30243.750 -61496.875   
3599              2.1625    -1.525    -7.775          -30243.750 -61496.875   

                     time  
      6BU8_6CU8            
0         0.000  18:00:00  
1    -50000.000  18:00:01  
2    -49996.875  18:00:02  
3    -49991.875  18:00:03  
4    -44986.875  18:00:04  
...         ...       ...  
3595 -50006.875  18:59:55  
3596 -50006.875  18:59:56  
3597 -50006.875  18:59:57  
3598 -50006.875  18:59:58  
3599 -50006.875  18:59:59  

[3600 rows x 7 columns]

# Spread verisinin grafiği çiziliyor
Örnekte yalnızca 18:00:00 - 19:00:00 arası kullanıldı

In [23]:
spread_18 = spread[0]

In [24]:
spread_18

2018-09-06 00:00:00                     2018-09-07 00:00:00             \
               6AU8_6BU8 6AU8_6CU8 6BU8_6CU8           6AU8_6BU8  6AU8_6CU8   
0                 0.0000     0.000     0.000               0.000      0.000   
1                 0.3500     0.350     0.000           45137.370  13887.370   
2                -4.6500    -1.525     3.125           81097.370  49850.495   
3                -8.3375    -1.525     9.025           81818.545  50574.795   
4             -3698.9625    -1.525  5914.025           -7269.580 -35385.205   
...                  ...       ...       ...                 ...        ...   
1717             18.6250     1.875   -24.925          -30231.350 -61496.975   
1718             18.6250     1.875   -24.925          -30231.350 -61496.975   
1719             18.6250     1.875   -24.925          -30231.350 -61496.975   
1720             18.6250     1.875   -24.925          -30231.350 -61496.975   
1721             18.6250     1.875   -24.925          -30231.350 -61496.975   

                     time  
      6BU8_6CU8            
0         0.000  18:00:00  
1    -50000.000  18:00:01  
2    -49996.875  18:00:02  
3    -49991.875  18:00:03  
4    -44986.875  18:00:04  
...         ...       ...  
1717 -50023.125  18:28:37  
1718 -50023.125  18:28:38  
1719 -50023.125  18:28:39  
1720 -50023.125  18:28:40  
1721 -50023.125  18:28:41  

[1722 rows x 7 columns]

In [27]:
graph.spread_scatter(spread_18)

# Tüm Günler Grafiği

In [28]:
spread_18 = spread[0]

In [29]:
AB = aux.extract_product(spread_18,'6AU8_6CU8')

In [30]:
grafik = graph.scatter_all_days(AB)

In [31]:
grafik.show()

----

# Change Hesaplanıyor

In [32]:
pairs = [pair[0]+'_'+pair[1] for pair in paired_products]

In [33]:
def change_al(df):
    change_dict = {}
    for date in non_duplicate_dates:
        for pair in pairs:
            sub_df = df[date][pair]            
            change_dict[(date,pair)] = aux.find_change(sub_df)
    new_df = pd.DataFrame(change_dict)
    new_df['time'] = df.time
    return new_df

In [34]:
change_list = [change_al(df) for df in spread]

---

# Duration ve Amplitude Hesaplanıyor

Hesaplama ile birlikte birleştirme de yapılıyor.
Birleştirme koşulu: <br/>
1. Ürünler aynı olacak
2. Tarihler farklı olacak
3. Saat dilimi aynı olacak

In [35]:
def _get_duramp(df):
    lengt = df.shape[1]
    df_list = []
    for i in range(lengt):
        df_list.append(aux.find_duramp(df.iloc[:,i]))
    return pd.concat(df_list).dropna()

In [36]:
def get_duramp(df):
    new_df = duzenle(df)
    product = {}
    for pair in pairs:
        product[pair] = _get_duramp(new_df[pair])
    return product

In [37]:
def duzenle(df):
    new_df = df.set_index('time')
    new_df = new_df.T.reset_index(level=0,drop=True).T
    return new_df

In [38]:
duramp = [get_duramp(change) for change in change_list]

---

# Koşula göre verileri yeniden düzenliyoruz

Veri yeniden düzenlenirken, 2 çeşit koşul koyma yöntemi kullanılır.
1. Medyan : <br/>
```py
con.single_scan(pandas.DataFrame, condition= 'median')
```
2. Percentile:
```py
con.single_scan(DataFrame,condition='percentile',quantile_value=  0<quantile_value<1)
```

**pozitif ve negatif medyan, duration ve amplitude için ayrı ayrı baz alınarak hesaplanmıştır**
```py
>>> edited_data.keys()
Out[]: dict_keys(['6AU8_6BU8', '6AU8_6CU8', '6BU8_6CU8'])
```
Bir ürüne ait tüm veriye erişmek için:
```py
edited_data['6AU8_6BU8']
```
Bir ürünün belirli bir saatine ait veriye erişmek için:
```py
edited_data['6AU8_6BU8'].loc[time(18)]
```


In [39]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

In [40]:
edited_data = {}

In [41]:
unconditional = {}

In [42]:
# condition='percentile' iken 
percentile_data = {}

In [44]:
for pair in pairs:
    con_list = []
    #con_percent_list=[]
    uncon_list = []
    for i in range(23):
        current = duramp[i][pair]
        con_df = con.single_scan(current)
        #con_df_percent = con.single_scan(current,condition='percentile',quantile_value=.25)
        con_list.append(con_df)
        uncon_list.append(current)
        #con_percent_list.append(con_df_percent)
    edited_data[pair] = pd.concat(con_list,keys=hour_series)
    unconditional[pair] = pd.concat(uncon_list,keys= hour_series)
    #percentile_data[pair] = pd.concat(con_percent_list,keys= hour_series)

---

# Dosyaya Yazılıyor

In [ ]:
for pair in pairs:
        print("dosyaya yazılıyor: ",pair)
        df = edited_data[pair].reset_index()
        uncon_df = unconditional[pair].reset_index()
        df.columns=['time_period','time','duration','amplitude']
        uncon_df.columns=['time_period','time','duration','amplitude']
        df.to_csv(pair+".csv")
        uncon_df.to_csv(pair+"_unconditional.csv",index = False)